In [27]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import HistGradientBoostingRegressor, HistGradientBoostingClassifier

import joblib


# -------------------- helpers --------------------
def summarize(name: str, arr):
    arr = np.asarray(arr, dtype=float)
    print(f"\n{name} summary:")
    print("  min:", float(np.min(arr)))
    print("  p50:", float(np.percentile(arr, 50)))
    print("  p90:", float(np.percentile(arr, 90)))
    print("  p99:", float(np.percentile(arr, 99)))
    print("  max:", float(np.max(arr)))


def tail_metrics(name: str, y_true: np.ndarray, y_pred: np.ndarray, thresholds=(3.0, 5.0, 10.0)):
    print(f"\n{name} tail metrics:")
    for t in thresholds:
        mask = y_true >= t
        n = int(mask.sum())
        if n < 5:
            print(f"  y >= {t}: n={n} (too few)")
            continue
        mae = mean_absolute_error(y_true[mask], y_pred[mask])
        r2 = r2_score(y_true[mask], y_pred[mask])
        print(f"  y >= {t}: n={n} | MAE={mae:.3f} | R2={r2:.4f}")


def _prep_features(df: pd.DataFrame) -> pd.DataFrame:
    X = df.copy()
    # Drop known non-feature cols if present
    X = X.drop(
        columns=[c for c in ["continent", "bmi_units", "scanner",
                             "mof_risk", "hip_risk"] if c in X.columns],
        errors="ignore"
    )

    # Normalize and one-hot encode us_group
    if "us_group" in X.columns:
        X["us_group"] = X["us_group"].astype(str).str.strip()
        # remove any pre-dummified columns to avoid duplicates
        X = X.drop(columns=[c for c in X.columns if c.startswith(
            "us_group_")], errors="ignore")
        X = pd.get_dummies(X, columns=["us_group"], drop_first=False)

    # Coerce numeric (Excel/CSV might have strings)
    X = X.apply(pd.to_numeric, errors="coerce")
    return X


# -------------------- TRAIN + SAVE --------------------
def train_and_save_fraxplus_models(
    df: pd.DataFrame,
    *,
    model_path: str = "fraxplus_models.pkl",
    test_size: float = 0.2,
    random_state: int = 42,
    hip_high_threshold: float = 2.0,
    eval_clamp: bool = False,
    prod_clamp_max: float = 100.0,
    tail_thresholds=(3.0, 5.0, 10.0),
):

    # ---- Build aligned dataset ONCE (prevents label/feature mismatch) ----
    base = df.drop(columns=[c for c in [
                   "continent", "bmi_units", "scanner"] if c in df.columns], errors="ignore").copy()
    base = base.dropna(subset=["mof_risk", "hip_risk"]).copy()
    if "us_group" in base.columns:
        base = base.dropna(subset=["us_group"]).copy()

    y_mof = base["mof_risk"].astype(float).to_numpy()
    y_hip = base["hip_risk"].astype(float).to_numpy()

    X = _prep_features(base)  # one-hot us_group + numeric coercion

    # ---- Split ----
    X_train, X_test, y_mof_train, y_mof_test, y_hip_train, y_hip_test = train_test_split(
        X, y_mof, y_hip, test_size=test_size, random_state=random_state
    )

    # ---- MOF model ----
    mof_model = HistGradientBoostingRegressor(
        learning_rate=0.05,
        max_iter=2000,
        l2_regularization=0.5,
        early_stopping=True,
        random_state=random_state,
        max_depth=4,
        min_samples_leaf=20,
    )
    print("\nTraining MOF (log1p)... {'max_depth': 4, 'min_samples_leaf': 20}")
    mof_model.fit(X_train, np.log1p(y_mof_train))

    mof_pred_t = mof_model.predict(X_test)
    mof_pred = np.expm1(mof_pred_t)

    # ---- HIP two-stage models ----
    print(f"Training Hip classifier (hip_risk >= {hip_high_threshold})...")
    y_high = (y_hip_train >= hip_high_threshold).astype(int)

    hip_clf = HistGradientBoostingClassifier(
        learning_rate=0.05,
        max_iter=2000,
        l2_regularization=0.5,
        early_stopping=True,
        random_state=random_state,
        max_depth=3,
        min_samples_leaf=40,
    )
    hip_clf.fit(X_train, y_high)

    low_mask = y_hip_train < hip_high_threshold
    high_mask = ~low_mask
    print(f"Training Hip regressor LOW (n={int(low_mask.sum())})...")
    print(f"Training Hip regressor HIGH (n={int(high_mask.sum())})...")

    hip_reg_low = HistGradientBoostingRegressor(
        learning_rate=0.05,
        max_iter=3000,
        l2_regularization=0.5,
        early_stopping=True,
        random_state=random_state,
        max_depth=3,
        min_samples_leaf=40,
    )
    hip_reg_high = HistGradientBoostingRegressor(
        learning_rate=0.03,
        max_iter=4000,
        l2_regularization=0.2,
        early_stopping=True,
        random_state=random_state,
        max_depth=4,
        min_samples_leaf=15,
    )

    hip_reg_low.fit(X_train[low_mask], np.log1p(y_hip_train[low_mask]))
    hip_reg_high.fit(X_train[high_mask], np.log1p(y_hip_train[high_mask]))

    # ---- HIP prediction in log space ----
    p_high = hip_clf.predict_proba(X_test)[:, 1]
    hip_low_t = hip_reg_low.predict(X_test)
    hip_high_t = hip_reg_high.predict(X_test)
    hip_pred_t = (1.0 - p_high) * hip_low_t + \
        p_high * hip_high_t  # log1p space

    # ---- HIP calibration in LOG space ----
    p_high_tr = hip_clf.predict_proba(X_train)[:, 1]
    low_tr_t = hip_reg_low.predict(X_train)
    high_tr_t = hip_reg_high.predict(X_train)
    hip_pred_tr_t = (1.0 - p_high_tr) * low_tr_t + \
        p_high_tr * high_tr_t

    y_hip_train_t = np.log1p(y_hip_train)
    Xcal = np.vstack([np.ones_like(hip_pred_tr_t), hip_pred_tr_t]).T
    a, b = np.linalg.lstsq(Xcal, y_hip_train_t, rcond=None)[0]
    a, b = float(a), float(b)
    print(f"\nHip calib (LOG space) a={a:.6f}, b={b:.6f}")

    # Apply calibration + CLIP in log space to avoid blow-ups
    hip_pred_t_cal = a + b * hip_pred_t
    hip_pred_t_cal = np.clip(hip_pred_t_cal, 0.0, np.log1p(prod_clamp_max))
    hip_pred = np.expm1(hip_pred_t_cal)

    # Optional: clamp MOF for evaluation
    if eval_clamp:
        mof_pred = np.clip(mof_pred, 0.0, prod_clamp_max)
        hip_pred = np.clip(hip_pred, 0.0, prod_clamp_max)

    # ---- Print metrics ----
    print("\n=== PERFORMANCE (holdout) ===")
    print(
        f"MOF  original: MAE={mean_absolute_error(y_mof_test, mof_pred):.3f} | "
        f"R2={r2_score(y_mof_test, mof_pred):.4f}"
    )
    print(
        f"MOF  log1p:    MAE={mean_absolute_error(np.log1p(y_mof_test), mof_pred_t):.3f} | "
        f"R2={r2_score(np.log1p(y_mof_test), mof_pred_t):.4f}"
    )

    # Hip metrics
    hip_true_t = np.log1p(y_hip_test)
    print(
        f"Hip  original (cal): MAE={mean_absolute_error(y_hip_test, hip_pred):.3f} | "
        f"R2={r2_score(y_hip_test, hip_pred):.4f}"
    )
    print(
        f"Hip  log1p    (cal): MAE={mean_absolute_error(hip_true_t, hip_pred_t_cal):.3f} | "
        f"R2={r2_score(hip_true_t, hip_pred_t_cal):.4f}"
    )

    tail_metrics("MOF", y_mof_test, mof_pred, thresholds=tail_thresholds)
    tail_metrics("Hip (cal)", y_hip_test, hip_pred, thresholds=tail_thresholds)

    # Debug summaries
    summarize("y_hip_test", y_hip_test)
    summarize("hip_pred", hip_pred)

    # ---- Save bundle ----
    bundle = {
        "mof_model": mof_model,
        "hip_clf": hip_clf,
        "hip_reg_low": hip_reg_low,
        "hip_reg_high": hip_reg_high,
        "feature_columns": list(X.columns),
        "hip_high_threshold": float(hip_high_threshold),
        "hip_calib_a": a,
        "hip_calib_b": b,
        "hip_calib_log_clip_max": float(prod_clamp_max),  # 100 by default
        "schema_version": 2,
    }
    joblib.dump(bundle, model_path)
    print(f"\nModels saved to {model_path}")
    return bundle

In [28]:
data = pd.read_csv('fraxdata.csv')

In [29]:
train_and_save_fraxplus_models(data)


Training MOF (log1p)... {'max_depth': 4, 'min_samples_leaf': 20}
Training Hip classifier (hip_risk >= 2.0)...
Training Hip regressor LOW (n=215)...
Training Hip regressor HIGH (n=185)...

Hip calib (LOG space) a=-0.014073, b=1.016230

=== PERFORMANCE (holdout) ===
MOF  original: MAE=0.799 | R2=0.9569
MOF  log1p:    MAE=0.093 | R2=0.9581
Hip  original (cal): MAE=0.414 | R2=0.9730
Hip  log1p    (cal): MAE=0.123 | R2=0.9560

MOF tail metrics:
  y >= 3.0: n=76 | MAE=0.915 | R2=0.9459
  y >= 5.0: n=58 | MAE=1.122 | R2=0.9259
  y >= 10.0: n=29 | MAE=1.474 | R2=0.8630

Hip (cal) tail metrics:
  y >= 3.0: n=34 | MAE=0.658 | R2=0.9596
  y >= 5.0: n=18 | MAE=0.854 | R2=0.9506
  y >= 10.0: n=4 (too few)

y_hip_test summary:
  min: 0.0
  p50: 1.6
  p90: 6.1
  p99: 18.0
  max: 18.0

hip_pred summary:
  min: 0.0
  p50: 1.257778520564137
  p90: 7.077303455642447
  p99: 18.287632325419853
  max: 19.914419458722204

Models saved to fraxplus_models.pkl


{'mof_model': HistGradientBoostingRegressor(early_stopping=True, l2_regularization=0.5,
                               learning_rate=0.05, max_depth=4, max_iter=2000,
                               random_state=42),
 'hip_clf': HistGradientBoostingClassifier(early_stopping=True, l2_regularization=0.5,
                                learning_rate=0.05, max_depth=3, max_iter=2000,
                                min_samples_leaf=40, random_state=42),
 'hip_reg_low': HistGradientBoostingRegressor(early_stopping=True, l2_regularization=0.5,
                               learning_rate=0.05, max_depth=3, max_iter=3000,
                               min_samples_leaf=40, random_state=42),
 'hip_reg_high': HistGradientBoostingRegressor(early_stopping=True, l2_regularization=0.2,
                               learning_rate=0.03, max_depth=4, max_iter=4000,
                               min_samples_leaf=15, random_state=42),
 'feature_columns': ['age',
  'sex_female',
  'weight_kg',
  'heigh